# Build Treasury Quotes CRSP

### CRSP Treasury Manual

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Treasury_Database_Guide_for_SAS_ASCII_EXCEL_R.pdf

https://www.crsp.org/products/documentation/treasno-properties

# WRDS API

In [44]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [45]:
import pandas as pd
import numpy as np
from datetime import date

***

# Database quote

In [46]:
libs = db.list_libraries()
libs_crsp = [item for item in libs if item.startswith('crsp')]
treasury_databases = db.list_tables('crsp_m_treasuries')

In [47]:
DATE = '2024-10-31'
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where caldt=\'{DATE}\''
data_quotes = db.raw_sql(SQL_QUERY)
data_quotes

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,204083.0,20241115.107500,2024-10-31,100.114004,100.114405,100.114204,M,C,3.444293,0.000141,0.000123,15.000000,3478.0,9604.0,0.0,NaN,None
1,204093.0,20281115.105250,2024-10-31,104.074219,104.121094,104.097656,M,C,2.411005,-0.000086,0.000112,1321.452576,7738.0,10947.0,0.0,NaN,None
2,204100.0,20370515.105000,2024-10-31,106.785156,106.894531,106.839844,M,C,2.296196,0.000662,0.000116,3424.760986,6423.0,21413.0,0.0,NaN,None
3,204102.0,20380515.104500,2024-10-31,101.562500,101.656250,101.609375,M,C,2.066576,0.001099,0.000118,3691.050841,7650.0,25500.0,0.0,NaN,None
4,206058.0,20390515.104250,2024-10-31,98.343750,98.421875,98.382812,M,C,1.951766,0.001441,0.000119,3923.457029,11634.0,38779.0,0.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,208285.0,20250327.400000,2024-10-31,98.228229,98.232285,98.230257,M,C,0.000000,0.000230,0.000121,147.000000,NaN,70915.0,0.0,NaN,None
444,208286.0,20250403.400000,2024-10-31,98.147071,98.154296,98.150683,M,C,0.000000,0.000259,0.000121,154.000000,NaN,76489.0,0.0,NaN,None
445,208289.0,20290930.203500,2024-10-31,97.117188,97.148438,97.132812,M,C,0.298077,-0.000302,0.000112,1657.424681,70000.0,71445.0,0.0,NaN,None
446,208294.0,20250211.400000,2024-10-31,98.743133,98.745967,98.744550,M,C,0.000000,0.000168,0.000123,103.000000,NaN,64212.0,0.0,NaN,None


In [48]:
KEYS = tuple(data_quotes['kytreasno'])
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYS}'
data_iss = db.raw_sql(SQL_QUERY)
data_iss

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204083.0,20241115.107500,20241115.107500,912810ES,1994-05-15,2024-11-15,1.0,7.500,2.0,3.7500,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204084.0,20250215.107620,20250215.107620,912810ET,1995-02-15,2025-02-15,0.0,7.625,2.0,3.8125,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204085.0,20250815.106870,20250815.106870,912810EV,1995-08-15,2025-08-15,0.0,6.875,2.0,3.4375,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204086.0,20260215.106000,20260215.106000,912810EW,1996-02-15,2026-02-15,0.0,6.000,2.0,3.0000,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204087.0,20260815.106750,20260815.106750,912810EX,1996-08-15,2026-08-15,0.0,6.750,2.0,3.3750,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,208302.0,20261031.204120,20261031.204120,91282CLS,2024-10-31,2026-10-31,0.0,4.125,2.0,2.0625,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
444,208303.0,20271015.203870,20271015.203870,91282CLQ,2024-10-15,2027-10-15,0.0,3.875,2.0,1.9375,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
445,208304.0,20291015.C01620,20291015.C01620,91282CLV,2024-10-15,2029-10-15,0.0,1.625,2.0,0.8125,...,None,0.0,None,12.0,0.0,1.0,1.0,None,None,None
446,208305.0,20291031.204120,20291031.204120,91282CLR,2024-10-31,2029-10-31,0.0,4.125,2.0,2.0625,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None


In [49]:
rawdata = data_quotes.merge(data_iss, on='kytreasno')
rawdata

,kytreasno,kycrspid_x,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204083.0,20241115.107500,2024-10-31,100.114004,100.114405,100.114204,M,C,3.444293,0.000141,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204093.0,20281115.105250,2024-10-31,104.074219,104.121094,104.097656,M,C,2.411005,-0.000086,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204100.0,20370515.105000,2024-10-31,106.785156,106.894531,106.839844,M,C,2.296196,0.000662,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204102.0,20380515.104500,2024-10-31,101.562500,101.656250,101.609375,M,C,2.066576,0.001099,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,206058.0,20390515.104250,2024-10-31,98.343750,98.421875,98.382812,M,C,1.951766,0.001441,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,208285.0,20250327.400000,2024-10-31,98.228229,98.232285,98.230257,M,C,0.000000,0.000230,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
444,208286.0,20250403.400000,2024-10-31,98.147071,98.154296,98.150683,M,C,0.000000,0.000259,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None
445,208289.0,20290930.203500,2024-10-31,97.117188,97.148438,97.132812,M,C,0.298077,-0.000302,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
446,208294.0,20250211.400000,2024-10-31,98.743133,98.745967,98.744550,M,C,0.000000,0.000168,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None


***

# Process Data

In [50]:
import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import process_wrds_treasury_data, select_maturities

data = process_wrds_treasury_data(rawdata)

In [51]:
idx = select_maturities(data)
data_select = data.loc[idx]

***

# Save to Excel

In [52]:
rawdata.columns.str.upper()

Index(['KYTREASNO', 'KYCRSPID_X', 'CALDT', 'TDBID', 'TDASK', 'TDNOMPRC',
       'TDNOMPRC_FLG', 'TDSOURCR', 'TDACCINT', 'TDRETNUA', 'TDYLD', 'TDDURATN',
       'TDPUBOUT', 'TDTOTOUT', 'TDPDINT', 'TDIDXRATIO', 'TDIDXRATIO_FLG',
       'KYCRSPID_Y', 'CRSPID', 'TCUSIP', 'TDATDT', 'TMATDT', 'IWHY', 'TCOUPRT',
       'TNIPPY', 'TVALFC', 'TFCPDT', 'IFCPDTF', 'TFCALDT', 'TNOTICE', 'IYMCN',
       'ITYPE', 'IUNIQ', 'ITAX', 'IFLWR', 'TBANKDT', 'TSTRIPELIG', 'TFRGNTGT'],
      dtype='object')

In [53]:
### raw data
outfile = f'../data/treasury_quotes_crsp_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    rawdata.columns = rawdata.columns.str.upper()
    rawdata.to_excel(writer, sheet_name= 'quotes', index=False)


### processed data
outfile = f'../data/treasury_quotes_clean_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    data.to_excel(writer, sheet_name= 'quotes', index=True)
    data_select.to_excel(writer, sheet_name = 'selected quotes', index=True)